# Modelowanie na danych obrazowych
Przechodzimy do części zajmującej się przygotowaniem kilku modeli oraz wybraniem zeń najlepszego kandydata do naszych finalnych rozważań dotyczących związanych z naszymi bazami danych. Rozważanymi przez nas architekturami będą autorsko tworzone architektury w framework'u keras. Oczywiście nie jest to koniecznie finalna architektura w przypadku całego projektu i w przypadku znalezienia lepszego rozwiązania dla naszego rozwiązania, zastosujemy je.

Przygotowywanie bibliotek.

In [44]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import PIL
from PIL import Image
import os
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

## Dane oraz cel modelu
Za pomocą naszych wcześniej wytworzonych narzędzi przygotowaliśmy jeden zbiór danych złożonych z danych pochodzących z następujących zbiorów danych:
- covidx-cxr2
- qatacov19-dataset
- siim-acr-pneumothorax-segmentation
- Shenzhen 
- rsna-pneumonia-detection-challenge 
Dane zostały podzielone na zbiory testowy, treningowy oraz walidacyjny (walidacyjny nie został ujęty w części treningowej).  
Dane we wszystkich zbiorach zostały jednakowo przetworzone i ustandaryzowane wedle narzędzi zaopatrzonych w pracy domowej 4.  
Celem naszego utworzonego zadania będzie klasyfikacja wielo-klasowa polegająca na predykcji choroby na podstawie skanu roentgena klatki piersiowej.  
W naszej bazie zostały zawarte następujące klasy z następująco przypisanymi im encodingami:  
- 0 - zdrowy
- 1 - covid 
- 2 - tuberculosis 
- 3 - pneumonia
Naszym zadaniem będzie poprawnie sklasyfikować daną etykiete.  
W zbiorze danych nie zawarliśmy wszystkich używanych przez nas baz danych ze względu na zbyt duże różnice między nimi by móc jakkolwiek logicznie dokonać ich konkatenacji i uczenia (np. jeden zbiór z danymi był tomografią komputerową podczas gdy reszta danych była skanami roentgena więc ich łączenie nie miałoby sensu).  
 **Uwaga** Dane były zrównoważone, tudzież jest tyle samo danych z każdej klasy, zatem użycie miary accuracy w dalszych rozważaniach będzie adekwatne.

In [2]:
labels = pd.read_csv(r'C:\Users\aaf6\Desktop\WB-2\Data_full\etykiety.csv')
labels = labels['class']

In [3]:
#Wgranie i przygotowanie danych
data = []
names = []
PATH = r'C:\Users\aaf6\Desktop\WB-2\Data_full\dataformodel'
files = Path(PATH).glob('*')
for file in files:
    names.append(str(file).split('\\')[-1])
    image = Image.open(file)
    data.append(np.asarray(image))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

Poniżej zamieszczono pobranie danych za pomocą wbudowanego modułu kerasowego, wymagało to przydzielenie instancji klas do osobnych folderów.

In [39]:
train = ImageDataGenerator()
train_dataset = train.flow_from_directory(r"C:\\Users\\aaf6\\Desktop\\WB-2\\Data_full_train\\")

Found 1344 images belonging to 4 classes.


In [40]:
test = ImageDataGenerator()
test_dataset = train.flow_from_directory(r"C:\\Users\\aaf6\\Desktop\\WB-2\\Data_full_test\\")

Found 400 images belonging to 4 classes.


### Tworzenie modeli
Poniżej dokonaliśmy utworzenia 5 testowych modeli za pomocą framework'u keras. Każdy zeń różni się nieznacznie jeżeli chodzi o złożoność oraz ilość warstw, jednakże każdy opiera się na podobnej architekturze naprzemiennych warstw konwolucyjnych oraz Pooling. W celu zabezpieczenia przed przeuczeniem zawarliśmy również warstwy *Dropout* w niektórych modelach.

Pierwszy model był naszym testowym, "domyślnym" o "sensownej" wielkości. Architektura zawarta została poniżej.  
W kerasie możliwym jest stworzenie modelu poprzez podawanie naprzemiennie warstw jakie chcemy ująć w naszsej architekturze.

In [31]:
model = keras.Sequential(
    [
        layers.Conv2D(16, (3,3), strides=2, activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.15),
        layers.Dense(4, activation="softmax")
    ]
)


Nasz drugi model został powiększony o dodatkowe dwa zestawy warstw konwolucyjnych oraz pooling. Ponieważ zdjęcia w naszej bazie są dość dużej rozdzielczości, użycie głębszej sieci może okazać się bardziej optymalne.

In [34]:
model_2 = keras.Sequential(
    [
        layers.Conv2D(16, (3,3), strides=2, activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.15),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.15),
        layers.Dense(4, activation="softmax")
    ]
)


Niewielkie różnice względem pierwszego modelu, zmienione zostały wartości okna w poolingu.

In [36]:
model_3 = keras.Sequential(
    [
        layers.Conv2D(16, (3,3), strides=2, activation="relu"),
        layers.MaxPooling2D(pool_size=(4, 4)),
        layers.Dropout(0.15),
        layers.Conv2D(32, (4,4), activation="relu"),
        layers.MaxPooling2D(pool_size=(4, 4)),
        layers.Dropout(0.1),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.15),
        layers.Dense(4, activation="softmax")
    ]
)

Poniższy cechuje się różnymi wartościami okien w warstwach konwolucyjnych oraz ilości neuronów w warstwie gęstej.

In [64]:
model_4 = keras.Sequential(
    [
        layers.Conv2D(16, (3,3), strides=2, activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(4, 4)),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(512,activation = 'relu'),
        layers.Dense(4, activation="softmax")
    ]
)

I ostatnia nasza architektura, cechująca się ponownie większą głębokością, lecz o innej wielkości okna w warstwie poolingu (2x2) oraz większej głębokości niż "domyślna" architektura.

In [71]:
model_5 = keras.Sequential(
    [
        layers.Conv2D(16, (3,3), strides=2, activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(512,activation = 'relu'),
        layers.Dense(4, activation="softmax")
    ]
)

## Etap uczenia
Poniżej przechodzimy do etapu uczenia naszych modeli na treningowej bazie danych. Treningu dokonaliśmy dla każdego z modeli na podobnej konfiguracji (optimizer RMSPROP oraz funkcja starty categorical_crossentropy przy batch size wynoszącym 32).  W ogólności przez charakter funkcjonowania warstw konwolucyjnych, rozdzielczość i sama ilość zdjęć, czas uczenia jest dość długi, chociaż nie przekracza 1 godziny.

In [33]:
model.compile(optimizer="rmsprop",metrics=['accuracy'],loss='categorical_crossentropy')
model.fit(x=train_dataset, batch_size=32, epochs=50)

Epoch 1/50
42/42 [==============================] - 27s 630ms/step - loss: 47.5339 - accuracy: 0.5246
Epoch 2/50
42/42 [==============================] - 26s 625ms/step - loss: 1.8403 - accuracy: 0.6897
Epoch 3/50
42/42 [==============================] - 27s 646ms/step - loss: 1.5047 - accuracy: 0.8021
Epoch 4/50
42/42 [==============================] - 28s 661ms/step - loss: 1.1573 - accuracy: 0.8408
Epoch 5/50
42/42 [==============================] - 27s 651ms/step - loss: 0.7600 - accuracy: 0.8713
Epoch 6/50
42/42 [==============================] - 29s 691ms/step - loss: 0.4510 - accuracy: 0.8951
Epoch 7/50
42/42 [==============================] - 28s 673ms/step - loss: 0.6834 - accuracy: 0.8810
Epoch 8/50
42/42 [==============================] - 28s 669ms/step - loss: 0.2176 - accuracy: 0.9211
Epoch 9/50
42/42 [==============================] - 29s 682ms/step - loss: 0.4008 - accuracy: 0.9003
Epoch 10/50
42/42 [==============================] - 27s 630ms/step - loss: 0.4126 - accur

AttributeError: 'Sequential' object has no attribute 'precict'

Powyższy błąd wynikał z lietrówki, nie należy się nim przejmować, został naprawiony i nie wpływa na sieć, ani jej uczenie.

In [35]:
model_2.compile(optimizer="rmsprop",metrics=['accuracy'],loss='categorical_crossentropy')
model_2.fit(x=train_dataset, batch_size=32, epochs=20)

Epoch 1/20
42/42 [==============================] - 29s 646ms/step - loss: 11.9510 - accuracy: 0.3065
Epoch 2/20
42/42 [==============================] - 29s 700ms/step - loss: 1.5016 - accuracy: 0.3661
Epoch 3/20
42/42 [==============================] - 29s 686ms/step - loss: 1.2450 - accuracy: 0.4970
Epoch 4/20
42/42 [==============================] - 36s 870ms/step - loss: 0.9306 - accuracy: 0.6295
Epoch 5/20
42/42 [==============================] - 49s 1s/step - loss: 0.7584 - accuracy: 0.7195
Epoch 6/20
42/42 [==============================] - 47s 1s/step - loss: 0.6617 - accuracy: 0.7634
Epoch 7/20
42/42 [==============================] - 48s 1s/step - loss: 0.5751 - accuracy: 0.7894
Epoch 8/20
42/42 [==============================] - 46s 1s/step - loss: 0.4729 - accuracy: 0.8371
Epoch 9/20
42/42 [==============================] - 46s 1s/step - loss: 0.4254 - accuracy: 0.8579
Epoch 10/20
42/42 [==============================] - 46s 1s/step - loss: 0.3795 - accuracy: 0.8594
Epoch 

In [37]:
model_3.compile(optimizer="rmsprop",metrics=['accuracy'],loss='categorical_crossentropy')
model_3.fit(x=train_dataset, batch_size=32, epochs=20)

Epoch 1/20
42/42 [==============================] - 43s 934ms/step - loss: 29.0291 - accuracy: 0.4196
Epoch 2/20
42/42 [==============================] - 38s 904ms/step - loss: 1.4224 - accuracy: 0.5439
Epoch 3/20
42/42 [==============================] - 44s 1s/step - loss: 0.9090 - accuracy: 0.6994
Epoch 4/20
42/42 [==============================] - 42s 1s/step - loss: 0.7423 - accuracy: 0.7634
Epoch 5/20
42/42 [==============================] - 42s 1s/step - loss: 0.5227 - accuracy: 0.8147
Epoch 6/20
42/42 [==============================] - 42s 1s/step - loss: 0.4485 - accuracy: 0.8534
Epoch 7/20
42/42 [==============================] - 42s 993ms/step - loss: 0.3551 - accuracy: 0.8884
Epoch 8/20
42/42 [==============================] - 42s 1s/step - loss: 0.3665 - accuracy: 0.8839
Epoch 9/20
42/42 [==============================] - 43s 1s/step - loss: 0.2924 - accuracy: 0.9085
Epoch 10/20
42/42 [==============================] - 49s 1s/step - loss: 0.2793 - accuracy: 0.9189
Epoch 11/

In [65]:
model_4.compile(optimizer="rmsprop",metrics=['accuracy'],loss='categorical_crossentropy')
model_4.fit(x=train_dataset, batch_size=32, epochs=40)

Epoch 1/40
42/42 [==============================] - 45s 1s/step - loss: 20.4066 - accuracy: 0.5893
Epoch 2/40
42/42 [==============================] - 45s 1s/step - loss: 1.4358 - accuracy: 0.6815
Epoch 3/40
42/42 [==============================] - 42s 999ms/step - loss: 1.2692 - accuracy: 0.7463
Epoch 4/40
42/42 [==============================] - 42s 998ms/step - loss: 1.6438 - accuracy: 0.7902
Epoch 5/40
42/42 [==============================] - 42s 998ms/step - loss: 0.2696 - accuracy: 0.9115
Epoch 6/40
42/42 [==============================] - 40s 951ms/step - loss: 0.8702 - accuracy: 0.8757
Epoch 7/40
42/42 [==============================] - 42s 997ms/step - loss: 1.4060 - accuracy: 0.8750
Epoch 8/40
42/42 [==============================] - 42s 995ms/step - loss: 0.3786 - accuracy: 0.9115
Epoch 9/40
42/42 [==============================] - 41s 975ms/step - loss: 0.2147 - accuracy: 0.9360
Epoch 10/40
42/42 [==============================] - 44s 1s/step - loss: 0.5015 - accuracy: 0.89

In [72]:
model_5.compile(optimizer="rmsprop",metrics=['accuracy'],loss='categorical_crossentropy')
model_5.fit(x=train_dataset, batch_size=32, epochs=40)

Epoch 1/40
42/42 [==============================] - 44s 1s/step - loss: 1.3631 - accuracy: 0.5074
Epoch 2/40
42/42 [==============================] - 43s 1s/step - loss: 0.7478 - accuracy: 0.7507
Epoch 3/40
42/42 [==============================] - 43s 1s/step - loss: 0.5256 - accuracy: 0.8214
Epoch 4/40
42/42 [==============================] - 45s 1s/step - loss: 0.4060 - accuracy: 0.8430
Epoch 5/40
42/42 [==============================] - 49s 1s/step - loss: 0.2940 - accuracy: 0.9025
Epoch 6/40
42/42 [==============================] - 48s 1s/step - loss: 0.2511 - accuracy: 0.9085
Epoch 7/40
42/42 [==============================] - 48s 1s/step - loss: 0.2413 - accuracy: 0.9293
Epoch 8/40
42/42 [==============================] - 47s 1s/step - loss: 0.1917 - accuracy: 0.9397
Epoch 9/40
42/42 [==============================] - 47s 1s/step - loss: 0.1237 - accuracy: 0.9568
Epoch 10/40
42/42 [==============================] - 48s 1s/step - loss: 0.1370 - accuracy: 0.9509
Epoch 11/40
42/42 [

Jak można było zauważyć, dla każdej z architektury osiągana wartość accuracy była niezwykle wysoka, wręcz przyrównywała się do jedynki. Podejrzanym faktorem jest potencjalne przeuczenie sieci bądź zbyt wielkie różnice w zdjęciach pomiędzy różnymi bazami danych, z których pochodziły dane. Zastosowanie zbioru walidacyjnego będzie konieczne przy zastosowaniu i konfiguracji naszego ostatecznego modelu.

## Dokonywanie predykcji oraz analizowanie poprawności modeli na danych testowych
Na koniec dokonamy sprawdzenia miary accuracy (dane były zrównoważone) na danych testowych (również zbiór zrównoważony) dla poszczególnych modeli.

In [79]:
#Funkcja pomocnicza do dokonywania predykcji
def prepare_labels_test(model):
    labels_pred = model.predict(test_dataset)
    labels_predicted = []
    for i in range(labels_pred.shape[0]):
        labels_predicted.append(list(labels_pred[i]).index(max(labels_pred[i])))
    return labels_predicted

In [80]:
#model 1
labels = prepare_labels_test(model)
accuracy_score(test_dataset.classes, labels)

0.21

In [85]:
#model 2
labels = prepare_labels_test(model_2)
accuracy_score(test_dataset.classes, labels)

0.2325

In [86]:
#model 3
labels = prepare_labels_test(model_3)
accuracy_score(test_dataset.classes, labels)

0.2475

In [87]:
#model 4
labels = prepare_labels_test(model_4)
accuracy_score(test_dataset.classes, labels)

0.27

In [88]:
#model 5
labels = prepare_labels_test(model_5)
accuracy_score(test_dataset.classes, labels)

0.25

## Wnioski
Poniżej zamieszczam streszczone wnioski oraz uwagi dla naszych modeli, które będziemy musieli wziąć pod uwagę przy wyborze naszego ostatecznego modelu (i jego modyfikacji).  
- Niestety ale modele wykazywały się dość niską miarą accuracy na danych testowych (blisko zgadywania) oraz bardzo dużymi na zbiorze treningowym, co jest dość nadzwyczajne gdyż dane pochodziły z jednej próbki co oznacza, iż powiniśmy jeszcze raz sprawdzić (i być może wyeksplorować dane testowe i treningowe) dane
- W zbiorze treningowym dysponowaliśmy 1344 zdjęciami, co może być niewystarczającą ilością i doszło do zwyczajnego przeuczenia się, będziemy musieli zwiększyć ilość danych .
- należy wziąć pod uwagę skorzystanie z głębszej sieci, która lepiej wyekstraktuje cechy naszych danych, gdyż istnieje podejrzenie zbyt płytkiej architektury
- przydałoby się dokonać przetasowania danych (shuffle) by zoptymalizować uczenie